In [ ]:
import numpy as np
import mat73
from tensorflow.keras.models import load_model
import scipy.io
import matplotlib.pyplot as plt
from matplotlib import cm


In [ ]:
model_filename = '../Best Neural Nets/eeg_cnn_20.h5'
model = load_model(model_filename,  compile=False)

In [ ]:
model.summary()

#### Eval with noisy sim data 


In [ ]:
# topos were generated with matlab with file create_cnn_input.
sources_path = 'single_source'
snr = 20
eeg_topos = mat73.loadmat('./eval_sim_data/old_nn/{}/{}db/eeg_topos_noisy.mat'.format(sources_path,snr))['eeg_topos']
eeg_topos = eeg_topos.transpose(2,0,1)
predicted_sources = model.predict(eeg_topos).T
np.save('./eval_sim_data/old_nn/{}/{}db/predicted_sources.npy'.format(sources_path,snr),predicted_sources)

#### Eval with real data

In [ ]:
# A1999,A1974,A0206
subject='A0206'
real_data_file = './real_data/{}/eeg_topo_real.mat'.format(subject)
topos_eval =  scipy.io.loadmat(real_data_file)['Zi'] 

plt.contourf(topos_eval, cmap=cm.get_cmap('viridis'))
cbar = plt.colorbar()
plt.title('Topography')
plt.show()

topos_eval= np.expand_dims(topos_eval, axis=0)

predicted_sources = model.predict(topos_eval).T
np.save('./real_data/{}/pred_sources_old.npy'.format(subject),predicted_sources)

In [ ]:
topos_eval = mat73.loadmat('../../Downloads/one_two_sources/eeg_topos.mat')['eeg_topos']
topos_eval = topos_eval.transpose(2,0,1)

In [ ]:
predicted_sources = model.predict(topos_eval).T

In [ ]:
np.save('../../Downloads/pred_real.npy',predicted_sources)

### Eval with depth dataset

In [ ]:
import os
from tqdm import tqdm 
snr = 20
path_to_depth = './eval_sim_data/old_nn/depth/{}db/'.format(snr)

for depth_folder in tqdm(os.listdir(path_to_depth)):
    depth_folder_full = os.path.join(path_to_depth, depth_folder)
    topos_eval = mat73.loadmat(os.path.join(depth_folder_full,'eeg_topos.mat'))['eeg_topos']

    topos_eval = topos_eval.transpose(2,0,1)
    predicted_sources = model.predict(topos_eval).T
    np.save(os.path.join(depth_folder_full,'predicted_sources.npy'),predicted_sources)